In [ ]:
%run -i classes_lkl_prof_functions.ipynb

In [ ]:
test = lkl_prof('/home2/karwal/lkl_prof/v2_lcdm_test/', 'lcdm_cmb_bao_sne_', 'H0')

test.prof_incr = 0.1
test.prof_max = 73.5
test.prof_min = 63.
test.processes = 1

In [ ]:
test.mcmc()

In [ ]:
test.global_min()

In [ ]:
test.init_lkl_prof()

In [ ]:
test.run_lkl_prof()

## Analysis functions 

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
chis = test.sum_params(['chi2__CMB', 'chi2__SN', 'chi2__BAO'])
lkl_prof_param = test.full_lkl_prof_dict()[test.prof_param]

In [ ]:
plt.plot( lkl_prof_param , chis )
plt.ylabel(r'$\chi^2$')
plt.xlabel(r'$'+lkl_pro_yaml['params'][test.prof_param]['latex']+'$')
plt.show()

In [ ]:
prof_lkl_output = test.full_lkl_prof_dict()

for param in test.param_order:
    plt.plot( prof_lkl_output[test.prof_param], prof_lkl_output[param] )
    try:
        plt.ylabel(r'$'+lkl_pro_yaml['params'][param]['latex']+'$')
    except KeyError:
        plt.ylabel([param])
    plt.xlabel(r'$'+lkl_pro_yaml['params'][test.prof_param]['latex']+'$')
#     plt.ylabel(param)
#     plt.xlabel(test.prof_param)
    plt.show()